### 토큰화 코드 

In [2]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


### 토큰 아이디에서 벡터로 변환

In [3]:

import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 일반 Python 리스트나 NumPy 배열을 직접 입력할 수 없으며, PyTorch 텐서 형태로 변환해야 함. 
input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)

# unsqueeze(0) 을 통해서 배치 처리를 위한 차원을 추가함. 
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)

# shape 를 통해 텐서의 차원을 드러냄 
input_embeddings.shape

torch.Size([1, 5, 16])

### 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [6]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

querys

tensor([[[-0.2674,  0.2305,  0.5756, -0.7030,  0.2837, -0.9053,  0.5753,
           0.6657, -1.2195, -0.5005,  0.1554, -0.0896, -0.2492, -0.0015,
           0.8065, -0.7903],
         [ 0.1401, -0.2307,  0.2017,  0.1211, -0.1198, -0.0447,  0.2522,
          -0.5284, -0.2062, -0.5332, -0.3514,  0.3258,  0.7349,  0.0908,
          -0.2024, -0.2072],
         [ 0.5048, -0.5202,  0.3082,  0.8571, -0.4276,  0.3955,  0.1452,
           0.3539, -0.8008, -0.0667,  0.5983,  0.5549, -0.4966, -0.2434,
           0.2790, -0.6430],
         [-0.6061, -0.6545,  1.3273, -0.3269,  0.0348, -0.6458,  0.4469,
          -0.0658, -0.0821, -0.8154, -0.1225, -0.4850,  0.0793,  0.2438,
           0.1696, -0.5300],
         [-0.5524,  0.4983,  0.1634,  0.5285, -0.4336, -0.2930, -0.0177,
          -0.5617,  0.3320,  0.4480, -0.2690, -0.4716,  0.1827, -0.1311,
           0.0356, -0.2522]]], grad_fn=<ViewBackward0>)

### 스케일 점곱 방식의 어텐션

In [10]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	# querys의 마지막 차원 크기를 가져옴
    dim_k = querys.size(-1) # 16
    
    # 어텐션 스코어 계산, 키 행렬을 전치해서 곱하고, 스케일링 (그레디언트 안정성을 위해서)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
 
    # 소프트맥스를 적용
    weights = F.softmax(scores, dim=-1)
	
    # 가중치와 값을 곱해서 최종 어텐션 벡터를 계산
    return weights @ values

# 어텐션 메커니즘 테스트
attention_weights = compute_attention(querys, keys, values)
print("attention_weights: ", attention_weights)


attention_weights:  tensor([[[-4.0728e-02,  6.2884e-02,  8.4439e-02,  4.8278e-01,  1.2792e-01,
           3.8908e-01, -1.8602e-01, -1.3477e-01,  3.2630e-01,  3.6521e-01,
          -9.8511e-02, -2.5141e-01,  2.5594e-01, -1.5738e-02, -1.0915e-01,
          -6.6629e-02],
         [-3.1253e-02,  1.4749e-01,  7.8763e-02,  5.1056e-01,  1.1171e-01,
           3.5293e-01, -3.4189e-01, -1.2582e-01,  3.2494e-01,  4.1650e-01,
           3.2637e-02, -1.8003e-01,  1.6691e-01, -7.4642e-03, -4.7548e-02,
          -1.4196e-01],
         [-7.1765e-02,  6.9422e-02,  3.1493e-02,  4.8302e-01,  1.3351e-01,
           4.1274e-01, -2.1709e-01, -1.3505e-01,  3.2711e-01,  3.9952e-01,
          -1.0553e-01, -1.9786e-01,  2.5885e-01, -2.0225e-04, -7.4752e-02,
          -1.0750e-01],
         [-2.2770e-02,  6.9439e-02,  4.8640e-02,  4.5960e-01,  1.1847e-01,
           3.6570e-01, -2.4218e-01, -9.6323e-02,  3.1936e-01,  3.8622e-01,
          -3.8909e-02, -2.3431e-01,  2.2183e-01, -1.6116e-02, -6.0861e-02,
        

### 어텐션 연산을 수행하는 AttentionHead 클래스

In [12]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings

tensor([[[ 0.3081,  0.0012, -0.1428,  0.2628, -0.2195, -0.1456, -0.3789,
           0.3101, -0.6648, -0.0789,  0.2911,  0.5670, -0.1493, -0.1894,
           0.1200, -0.0425],
         [ 0.3322, -0.0071, -0.1866,  0.1682, -0.3243, -0.2008, -0.4458,
           0.1997, -0.6346, -0.0176,  0.2912,  0.4852, -0.0748, -0.0778,
           0.1154, -0.1495],
         [ 0.3692, -0.0417, -0.1578,  0.1285, -0.2881, -0.1605, -0.3935,
           0.1963, -0.7002, -0.0833,  0.2586,  0.5807, -0.1052, -0.1088,
           0.0769, -0.1117],
         [ 0.3183,  0.0032, -0.1456,  0.2193, -0.2920, -0.1711, -0.4009,
           0.2495, -0.6619, -0.0474,  0.2923,  0.5399, -0.1111, -0.1236,
           0.1099, -0.0994],
         [ 0.3631, -0.0420, -0.2001,  0.1284, -0.2629, -0.1775, -0.4371,
           0.2028, -0.6404, -0.0666,  0.2586,  0.5074, -0.0937, -0.1127,
           0.1029, -0.1225]]], grad_fn=<UnsafeViewBackward0>)

### 멀티 헤드 어텐션 구현

In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    # B: 배치 크기, T: 시퀀스 길이, C: 임베딩 차원
    B, T, C = querys.size()
    
    # view 로 재구성, 16차원을 4개의 헤드로 4차원식 나눔 
    # 멀티 헤드 어텐션에서는 그러면 차원을 여러개로 나눠서 각각의 차원에 대해 가중치 파라미터를 학습하는거임
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    
    # 각 헤드별로 독립적인 어텐션 계산 
    attention = compute_attention(querys, keys, values, self.is_causal)
    
    # 헤드별 어텐션 벡터를 연결하고, 최종 선형 변환
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape


### 층 정규화 코드

In [13]:

# 파이토치에서는 nn.LayerNorm 을 사용해서 정규화 코드를 만들 수 있음. 
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[-1.8626e-09,  4.0978e-08,  0.0000e+00,  7.4506e-09,  5.9605e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

### 피드 포워드 층 코드

In [ ]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x


### 인코더 층


In [14]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x


### 인코더 구현 


In [15]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output


### 디코더에서 어텐션 연산(마스크 어텐션)

In [16]:
def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
	if is_causal:
		query_length = querys.size(2)
		key_length = keys.size(2)
		temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
		scores = scores.masked_fill(temp_mask == False, float("-inf"))
	weights = F.softmax(scores, dim=-1) # (1, 5, 5)
	return weights @ values # (1, 5, 16)

### 크로스 어텐션이 포함된 디코더 층

In [ ]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x
